## Campaign success prediction
Analyze Adwords campaign text (Header, Description) and predict how successfull is going to be

In [75]:
import torch
from fastai.text import (
        load_learner,
        defaults
)
import pathlib as path
from IPython.display import display, Markdown, clear_output
from ipywidgets import Layout, Output, HBox, VBox, Button, Text, HTML, RadioButtons

In [4]:
# CPU for inference (think is default anyhow)
defaults.device = torch.device('cpu')

In [5]:
MODELS = path.Path('/home/jovyan/work/data/corpus/models')

In [6]:
learner_headline = load_learner(MODELS, 'headline-learner.pkl')
learner_description = load_learner(MODELS, 'description-learner.pkl')

In [76]:
from fastai.text import Tokenizer, SpacyTokenizer, Vocab
tokenizer = Tokenizer()

In [70]:
tok = SpacyTokenizer('hr')

In [77]:
vocab = Vocab.load(MODELS/'campaign_vocab.pkl')

In [86]:
vocab.numericalize("ekupi hladnjake dsadsD".split())

[0, 541, 0]

In [96]:
unknown_words_pct("ekupi hladnjake dsadsD")

0.6666666666666666

In [87]:
vocab.textify([0, 541, 0])

'xxunk hladnjake xxunk'

In [72]:
tokenizer.process_text("DFGD asdas dsadsadsa", tok )

['xxup', 'dfgd', 'asdas', 'dsadsadsa']

In [73]:
%%html
<style>
    .input_style {
        width: 30em;
    }
    .box_style {
        width:40px;
        height: 40px;
        border : 1px solid gray;
    }
    .box_color_1 {
        background-color: #FF0000;
    }
    .box_color_2 {
        background-color:#FF3300;
    }
    .box_color_3 {
        background-color: #FF6600;
    }
    .box_color_4 {
        background-color: #FF9900;
    }
    .box_color_5 {
        background-color: #FFCC00;
    }
    .box_color_6 {
       background-color: #FEFF00;
    }
    .box_color_7 {
       background-color: #CBFF00;
    }
    .box_color_8 {
       background-color: #98FF00;
    }
    .box_color_9 {
       background-color: #65FF00;
    }
    .box_color_10 {
       background-color: #32FF00;
    }
    .box_color_11 {
       background-color: rgb(0, 255, 0);
    }
</style>

In [130]:
input_text = Text(
       placeholder='Unesi tekst za predikciju',
       description='Tekst: '
)

input_text.add_class("input_style")

button = Button(
    description='Provjeri',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Provjeri',
    icon='arrow-right' # (FontAwesome names without the `fa-` prefix)
)

rbs = RadioButtons(
    options=['Headline', 'Description'],
#    value='Headline', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description='Type:',
    disabled=False
)

output = Output()

box1 = VBox([rbs])
box2 = VBox([input_text])
box3 = VBox([button])

def get_prediction_box(i, label, text):
    box1 = VBox(layout=Layout(justify_content= 'flex-end'))
    box1.add_class("box_style")
    box1.add_class(f'box_color_{i}')
    # if DEBUG:
    print(f'Box color class: {i}')
    html = HTML(
        value=f'<b>{text}</b>',
        placeholder=label,
        description=label,
    )
    box2 = VBox([html])
    return HBox([box1, box2])

out_box = HBox([box1, box2, box3])

def get_category(rate, part):
    ''' Find a color from Int. rate, scale from 1-11 (6 normal text)'''
    neutral = 9 if part == 'Description' else 7.5
    step = 1.4 if part == 'Description' else 1.2
    cat = round((rate - neutral)/step) + 6
    cat = cat if cat >0 else 1
    cat = cat if cat < 11 else 11
    return cat

def unknown_words_pct(text: str):
    vocab_codes = vocab.numericalize(text.split())
    return 100 * vocab_codes.count(0) / len(vocab_codes)
    
def check_text(b):
    with output:
        text = input_text.value.lower()
        if text != '':
            if rbs.value == 'Headline':
                rate, _, losses = learner_headline.predict(text)
                display(get_prediction_box(get_category(rate.data[0], 'Headline'),'Headline', input_text.value) )
            else:
                rate, _, losses = learner_description.predict(text)
                display(get_prediction_box(get_category(rate.data[0], part='Description'),'Description', input_text.value) )
            if DEBUG:
                print(f'Predicted interaction rate: {rate}')
                print(f'Percentage of unknown words: {unknown_words_pct(text)}%\n')
        else:
            print ('No text to predict!')
        print('\n')
        
button.on_click(check_text)

In [124]:
scale = '''
### Color representation of message sentiment
Greener the color, the better is interaction rate, more red it is, lower interaction rate is forecasted! 
Yellow class (6) represents text with neutral IR.
![Gradient](../images/gradient2.png)
'''

In [131]:
DEBUG=False 
output.clear_output()
# display(input_text)
# display(button, output)
display(Markdown(scale))
display(out_box)
display(output)


### Color representation of message sentiment
Greener the color, the better is interaction rate, more red it is, lower interaction rate is forecasted! 
Yellow class (6) represents text with neutral IR.
![Gradient](../images/gradient2.png)


Output()

In [ ]:
%debug